<a href="https://colab.research.google.com/github/gustavoh-dev/meus-projetos/blob/main/Sistema_Lavanderia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# --- 1. INSTALAÇÃO DE PACOTES E FONTES ---
print("⏳ Configurando ambiente... (aguarde uns segundos)")
!sudo apt-get update -qq
!sudo apt-get install wkhtmltopdf fonts-liberation -y -qq
!pip install pdfkit gradio -q
print("✅ Instalação concluída! Iniciando o sistema...")

import gradio as gr
import sqlite3
import random
import pdfkit
import csv
import os
from datetime import datetime

# --- 2. BANCO DE DADOS ---
def conectar():
    return sqlite3.connect('lavanderia.db')

def criar_tabelas():
    conn = conectar()
    cursor = conn.cursor()

    # Tabela Pedidos
    try: cursor.execute("SELECT pagamento FROM pedidos LIMIT 1")
    except: cursor.execute("DROP TABLE IF EXISTS pedidos")

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS pedidos (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            cliente TEXT NOT NULL,
            cpf TEXT NOT NULL,
            tipo_roupa TEXT NOT NULL,
            valor REAL NOT NULL,
            pagamento TEXT NOT NULL,
            data_entrada TEXT NOT NULL,
            estado TEXT DEFAULT 'Pendente'
        )
    ''')

    # Tabela Usuários
    try: cursor.execute("SELECT funcao FROM usuarios LIMIT 1")
    except: cursor.execute("DROP TABLE IF EXISTS usuarios")

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS usuarios (
            username TEXT PRIMARY KEY,
            senha TEXT NOT NULL,
            funcao TEXT NOT NULL
        )
    ''')

    cursor.execute("INSERT OR IGNORE INTO usuarios (username, senha, funcao) VALUES (?, ?, ?)",
                   ("admin", "1234", "admin"))

    conn.commit()
    conn.close()

criar_tabelas()

# --- 3. SISTEMA DE LOGIN ---
def verificar_credenciais(username, password):
    conn = conectar(); cursor = conn.cursor()
    cursor.execute("SELECT senha FROM usuarios WHERE username = ?", (username,))
    res = cursor.fetchone(); conn.close()
    if res and res[0] == password: return True
    return False

def verificar_permissao_admin(request: gr.Request):
    if request is None: return False
    user = request.username
    conn = conectar(); cursor = conn.cursor()
    cursor.execute("SELECT funcao FROM usuarios WHERE username = ?", (user,))
    res = cursor.fetchone(); conn.close()
    if res and res[0] == 'admin': return True
    return False

def cadastrar_novo_usuario(novo_user, nova_senha, funcao, req: gr.Request):
    if not verificar_permissao_admin(req): return "⛔ Acesso Negado.", listar_usuarios(req)
    if not novo_user or not nova_senha: return "⚠️ Preencha tudo.", listar_usuarios(req)
    conn = conectar(); cursor = conn.cursor()
    try:
        cursor.execute("INSERT INTO usuarios VALUES (?, ?, ?)", (novo_user, nova_senha, funcao))
        conn.commit(); msg = f"✅ Usuário {novo_user} criado!"
    except: msg = "❌ Usuário já existe."
    conn.close()
    return msg, listar_usuarios(req)

def excluir_usuario(alvo, req: gr.Request):
    if not verificar_permissao_admin(req): return "⛔ Acesso Negado.", listar_usuarios(req)
    if alvo == "admin": return "❌ Não pode excluir o admin principal.", listar_usuarios(req)
    conn = conectar(); cursor = conn.cursor()
    cursor.execute("DELETE FROM usuarios WHERE username = ?", (alvo,))
    conn.commit(); conn.close()
    return f"🗑️ {alvo} excluído.", listar_usuarios(req)

def listar_usuarios(req: gr.Request = None):
    admin = verificar_permissao_admin(req) if req else False
    conn = conectar(); cursor = conn.cursor()
    if admin: cursor.execute("SELECT username, senha, funcao FROM usuarios")
    else: cursor.execute("SELECT username, '******', funcao FROM usuarios")
    users = cursor.fetchall(); conn.close()
    return users

# --- 4. LÓGICA DO SISTEMA (COM BUSCA) ---

def pegar_dados_tabela(filtro=None):
    conn = conectar()
    cursor = conn.cursor()

    # SE TIVER FILTRO, FAZ UMA BUSCA ESPECÍFICA
    if filtro:
        termo = f"%{filtro}%" # O % permite buscar partes do nome
        cursor.execute("""
            SELECT id, data_entrada, cliente, cpf, tipo_roupa, valor, pagamento, estado
            FROM pedidos
            WHERE cliente LIKE ? OR cpf LIKE ?
        """, (termo, termo))
    else:
        # SE NÃO TIVER FILTRO, MOSTRA TUDO
        cursor.execute("SELECT id, data_entrada, cliente, cpf, tipo_roupa, valor, pagamento, estado FROM pedidos")

    return cursor.fetchall()

def nova_solicitacao(cliente, cpf, roupa, valor, pagamento):
    cpf = str(cpf).strip()
    if not cpf.isdigit() or len(cpf) != 11: return "⚠️ Erro CPF: Precisa ter 11 números.", pegar_dados_tabela()
    if not cliente or not roupa or valor <= 0 or not pagamento: return "⚠️ Preencha todos os campos.", pegar_dados_tabela()

    data_atual = datetime.now().strftime("%d/%m/%Y %H:%M")
    conn = conectar()
    cursor = conn.cursor()
    cursor.execute("INSERT INTO pedidos (cliente, cpf, tipo_roupa, valor, pagamento, data_entrada, estado) VALUES (?, ?, ?, ?, ?, ?, ?)",
                   (cliente, cpf, roupa, float(valor), pagamento, data_atual, 'Pendente'))
    conn.commit()
    conn.close()
    return f"✅ Pedido registrado!", pegar_dados_tabela()

def atualizar_status(id_pedido, novo_estado):
    try: id_pedido = int(id_pedido)
    except: return "❌ ID inválido", pegar_dados_tabela()
    conn = conectar(); cursor = conn.cursor()
    cursor.execute("SELECT id FROM pedidos WHERE id = ?", (id_pedido,))
    if not cursor.fetchone(): conn.close(); return "❌ ID não encontrado.", pegar_dados_tabela()
    cursor.execute("UPDATE pedidos SET estado = ? WHERE id = ?", (novo_estado, id_pedido))
    conn.commit(); conn.close()
    return f"✅ Pedido {id_pedido} atualizado.", pegar_dados_tabela()

def excluir_pedido(id_pedido, req: gr.Request):
    if not verificar_permissao_admin(req): return "⛔ Acesso Negado.", pegar_dados_tabela()
    try: id_pedido = int(id_pedido)
    except: return "❌ ID inválido", pegar_dados_tabela()
    conn = conectar(); cursor = conn.cursor()
    cursor.execute("DELETE FROM pedidos WHERE id = ?", (id_pedido,))
    conn.commit(); conn.close()
    return "🗑️ Excluído.", pegar_dados_tabela()

# --- 5. RELATÓRIOS ---
def gerar_relatorio_geral(req: gr.Request):
    if not verificar_permissao_admin(req): return "<h3>⛔ Acesso Negado</h3>", None
    conn = conectar(); cursor = conn.cursor()
    cursor.execute("SELECT * FROM pedidos"); dados = cursor.fetchall(); conn.close()
    if not dados: return "<h3>Sem dados.</h3>", None

    total_fat = 0; total_ped = len(dados)
    status_count = {"Pendente":0, "Lavando":0, "Pronto":0, "Concluído":0}
    nome_arq = "Relatorio_Completo.csv"

    with open(nome_arq, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f, delimiter=';')
        writer.writerow(["ID", "Data Entrada", "Cliente", "CPF", "Roupa", "Valor", "Pagamento", "Estado"])
        for p in dados:
            total_fat += p[4]; st = p[7]
            if st in status_count: status_count[st] += 1
            writer.writerow([p[0], p[6], p[1], p[2], p[3], f"{p[4]:.2f}".replace('.',','), p[5], p[7]])

    html = f"""
    <div style="font-family:'Roboto',Arial; color:#333;">
        <div style="display:flex; gap:20px; margin-bottom:20px;">
            <div style="background:#e0f7fa; padding:20px; border-radius:10px; width:50%; border:1px solid #b2ebf2;">
                <h3 style="margin:0; color:#006064;">💰 Faturamento</h3><p style="font-size:32px; font-weight:bold; margin:10px 0; color:#006064;">R$ {total_fat:.2f}</p>
            </div>
            <div style="background:#fff3e0; padding:20px; border-radius:10px; width:50%; border:1px solid #ffe0b2;">
                <h3 style="margin:0; color:#e65100;">📦 Pedidos</h3><p style="font-size:32px; font-weight:bold; margin:10px 0; color:#e65100;">{total_ped}</p>
            </div>
        </div>
        <ul><li>🔴 Pendente: {status_count.get('Pendente',0)}</li><li>🔵 Lavando: {status_count.get('Lavando',0)}</li><li>🟡 Pronto: {status_count.get('Pronto',0)}</li><li>🟢 Concluído: {status_count.get('Concluído',0)}</li></ul>
    </div>
    """
    return html, nome_arq

# --- 6. DANFE ---
def construir_html_danfe(id_pedido):
    try: id_pedido = int(id_pedido)
    except: return None, "ID Inválido"
    conn = conectar(); cursor = conn.cursor()
    cursor.execute("SELECT * FROM pedidos WHERE id = ?", (id_pedido,))
    pedido = cursor.fetchone(); conn.close()
    if not pedido: return None, "Pedido não encontrado"
    if pedido[7] != "Concluído": return None, pedido[7]

    data_ent = pedido[6]; pagto = pedido[5]; cpf, emissao = pedido[2], datetime.now().strftime("%d/%m/%Y")
    chave = f"2625 0412 3456 7800 0190 5500 1000 {random.randint(100000, 999999)} 1000 {random.randint(100000, 999999)}"
    num, val = f"{pedido[0]:09d}", f"{pedido[4]:.2f}"
    bar_css = "background: repeating-linear-gradient(to right, #000 0px, #000 2px, #fff 2px, #fff 4px, #000 4px, #000 5px, #fff 5px, #fff 7px);"

    html = f"""
    <!DOCTYPE html><html><head><meta charset="UTF-8">
    <link href="https://fonts.googleapis.com/css2?family=Roboto:wght@400;700;900&display=swap" rel="stylesheet">
    <style>body{{font-family:'Roboto',Arial!important;background:white}}#d{{background:#fff;padding:20px;max-width:800px;margin:auto;border:1px solid #ccc;color:#000!important}}#d *{{border-color:#000!important;color:#000!important}}.b{{border:1px solid #000}}.tp{{font-size:11px}}.tl{{font-size:10px;font-weight:700;text-transform:uppercase}}table{{width:100%;border-collapse:collapse}}td,th{{padding:5px}}.g{{background:#f0f0f0!important}}</style>
    </head><body><div id="d">
    <table class="b" style="margin-bottom:5px"><tr><td width="80%"><div class="tl">RECEBEMOS DE LAVANDERIA VASCONCELOS OS SERVIÇOS CONSTANTES NA NOTA</div></td><td width="20%" align="center" style="border-left:1px solid #000"><div class="tl">NF-e</div><div style="font-weight:900">Nº {num}</div></td></tr></table>
    <table class="b"><tr><td width="38%" valign="top" style="padding:10px"><div style="font-size:20px;font-weight:900">LAVANDERIA VASCONCELOS</div><div class="tp">Rua Escritor Alvaro Lins, 73<br>Várzea - Recife - PE<br>CEP: 50830-420<br><strong>Fone: (81) 98865-9273</strong></div></td><td width="20%" align="center" style="border-left:1px solid #000;border-right:1px solid #000"><div style="font-size:28px;font-weight:900">DANFE</div><div class="tp">Nota Fiscal Eletrônica</div><br><span class="tp">1 - Saída</span><span style="border:1px solid #000;padding:2px 6px;font-weight:900;margin-left:5px">1</span><br><br><div style="font-weight:900">Nº {num}</div></td><td width="42%" valign="top" style="padding:10px"><div style="height:50px;width:100%;{bar_css}margin-bottom:5px"></div><div class="tl">CHAVE DE ACESSO</div><div style="font-size:12px;background:#eee;text-align:center;border:1px solid #ccc">{chave}</div></td></tr></table>
    <table class="b" style="border-top:0"><tr><td width="60%"><div class="tl">NATUREZA DA OPERAÇÃO</div><div style="font-weight:700;font-size:12px">PRESTAÇÃO DE SERVIÇO</div></td><td width="40%"><div class="tl">PROTOCOLO</div><div class="tp">13523000{random.randint(100000,999999)} - {emissao}</div></td></tr></table>
    <div class="tl" style="margin-top:10px;background:#ccc;padding:3px;border:1px solid #000;border-bottom:0">DESTINATÁRIO</div>
    <table class="b"><tr><td width="60%"><div class="tl">NOME / RAZÃO SOCIAL</div><div style="font-weight:700;font-size:12px">{pedido[1]}</div></td><td width="25%"><div class="tl">CPF</div><div style="font-weight:700">{cpf}</div></td><td width="15%"><div class="tl">EMISSÃO</div><div>{emissao}</div></td></tr></table>
    <div class="tl" style="margin-top:10px;background:#ccc;padding:3px;border:1px solid #000;border-bottom:0">PRODUTOS</div>
    <table class="b"><tr class="g"><th class="b tl" align="left">DESCRIÇÃO</th><th class="b tl" align="right">VALOR TOTAL</th></tr><tr><td class="b tp" style="font-weight:700">LAVAGEM DE {pedido[3].upper()}</td><td class="b tp" align="right">{val}</td></tr></table>
    <table style="margin-top:10px"><tr><td width="70%" class="b" valign="top"><div class="tl">DADOS ADICIONAIS</div><div class="tp" style="margin-top:5px">Entrada: {data_ent}<br><strong>Pagamento: {pagto}</strong><br>Simples Nacional. Obrigado!</div></td><td width="30%" style="padding-left:10px"><div class="tl" align="right">VALOR TOTAL</div><div class="b g" style="padding:10px;text-align:right;font-size:18px;font-weight:900">R$ {val}</div></td></tr></table>
    </div></body></html>
    """
    return html, "Sucesso"

def processar_danfe(id_pedido):
    html, status = construir_html_danfe(id_pedido)
    if html is None:
        return f"""<div style="background:#fff3cd;padding:25px;border:2px solid #ffeeba;border-radius:8px;color:#856404!important"><h3 style="margin:0;color:#856404!important">⚠️ AÇÃO BLOQUEADA</h3><p style="color:#856404!important">Status atual: <strong>{status}</strong>. Mude para 'Concluído'.</p></div>""", None
    fname = f"DANFE_{id_pedido}.pdf"
    config = pdfkit.configuration(wkhtmltopdf='/usr/bin/wkhtmltopdf')
    opts = {'page-size':'A4','margin-top':'10mm','margin-right':'10mm','margin-bottom':'10mm','margin-left':'10mm','encoding':"UTF-8",'no-outline':None,'enable-local-file-access':None}
    try: pdfkit.from_string(html, fname, configuration=config, options=opts)
    except Exception as e: return f"Erro: {e}", None
    return html, fname

# --- 7. INTERFACE GRADIO ---
with gr.Blocks(theme=gr.themes.Soft()) as app:
    gr.Markdown("# 🧺 Sistema Lavanderia Vasconcelos")

    # --- BARRA DE BUSCA ---
    with gr.Row():
        txt_busca = gr.Textbox(label="🔍 Buscar Pedido", placeholder="Digite Nome ou CPF e aperte Enter")
        btn_busca = gr.Button("Buscar")

    # TABELA PRINCIPAL
    tabela_global = gr.Dataframe(
        headers=["ID", "Data", "Cliente", "CPF", "Roupa", "Valor", "Pagto", "Estado"],
        value=pegar_dados_tabela(), label="Quadro Geral", interactive=False
    )

    # AÇÕES DA BUSCA E ATUALIZAÇÃO
    btn_busca.click(pegar_dados_tabela, inputs=[txt_busca], outputs=tabela_global)
    txt_busca.submit(pegar_dados_tabela, inputs=[txt_busca], outputs=tabela_global) # Permite usar a tecla Enter
    gr.Button("🔄 Atualizar Tabela Completa").click(lambda: pegar_dados_tabela(None), outputs=tabela_global)

    with gr.Tabs():
        # NOVA SOLICITAÇÃO
        with gr.TabItem("➕ Nova Solicitação"):
            with gr.Row():
                n = gr.Textbox(label="Nome do Cliente")
                c = gr.Textbox(label="CPF", max_length=11)
            with gr.Row():
                r = gr.Textbox(label="Tipo de Roupa")
                v = gr.Number(label="Valor (R$)")
                p = gr.Dropdown(["Pix", "Cartão Crédito", "Cartão Débito", "Dinheiro"], label="Pagamento", value="Pix")
            gr.Button("Salvar Pedido", variant="primary").click(nova_solicitacao, [n,c,r,v,p], [gr.Markdown(), tabela_global])

        # ATUALIZAR
        with gr.TabItem("🔄 Atualizar Estado"):
            with gr.Row():
                i = gr.Number(label="ID do Pedido")
                s = gr.Dropdown(["Lavando", "Pronto", "Concluído"], label="Novo Estado")
            gr.Button("Atualizar").click(atualizar_status, [i,s], [gr.Markdown(), tabela_global])

        # RELATÓRIO
        with gr.TabItem("📊 Relatórios (Admin)"):
            gr.Button("Gerar Relatório").click(gerar_relatorio_geral, [], [gr.HTML(), gr.File()])

        # DANFE
        with gr.TabItem("📄 DANFE & PDF"):
            with gr.Row():
                i_nota = gr.Number(label="ID do Pedido")
                btn_danfe = gr.Button("Gerar DANFE e PDF")
            btn_danfe.click(processar_danfe, inputs=[i_nota], outputs=[gr.HTML(), gr.File()])

        # EXCLUIR
        with gr.TabItem("🗑️ Excluir (Admin)"):
            i_del = gr.Number(label="ID")
            gr.Button("Excluir", variant="stop").click(excluir_pedido, [i_del], [gr.Markdown(), tabela_global])

        # USUÁRIOS
        with gr.TabItem("👥 Usuários (Admin)"):
            with gr.Row():
                with gr.Column():
                    gr.Markdown("### Cadastrar Novo")
                    new_user = gr.Textbox(label="Usuário")
                    new_pass = gr.Textbox(label="Senha")
                    new_role = gr.Dropdown(["admin", "comum"], label="Nível", value="comum")
                    btn_cad_user = gr.Button("Cadastrar")
                with gr.Column():
                    gr.Markdown("### Excluir Usuário")
                    del_user = gr.Textbox(label="Nome do Usuário")
                    btn_del_user = gr.Button("Excluir Usuário", variant="stop")
            msg_user = gr.Markdown()
            btn_refresh_users = gr.Button("🔄 Ver Lista")
            lista_users = gr.Dataframe(headers=["Usuário", "Senha", "Função"], label="Base de Usuários")
            btn_cad_user.click(cadastrar_novo_usuario, [new_user, new_pass, new_role], [msg_user, lista_users])
            btn_del_user.click(excluir_usuario, [del_user], [msg_user, lista_users])
            btn_refresh_users.click(listar_usuarios, [], lista_users)

app.launch(auth=verificar_credenciais, share=True)

⏳ Configurando ambiente... (aguarde uns segundos)
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
✅ Instalação concluída! Iniciando o sistema...


/tmp/ipython-input-232384459.py:236: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft()) as app:


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c7dbaec54bb3ba392d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
